# Data Preparation

In [7]:
import os
import requests

def get_iwsltenvi_data(save: bool = True):
    train_en_url = 'https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.en'
    train_vi_url = 'https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/train.vi'
    test_en_url = 'https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/tst2013.en'
    test_vi_url = 'https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/tst2013.vi'

    train_en = requests.get(train_en_url).text
    train_vi = requests.get(train_vi_url).text
    test_en = requests.get(test_en_url).text
    test_vi = requests.get(test_vi_url).text

    store_pth = './data/iwsltenvi/'
    os.makedirs(store_pth, exist_ok=True)
    print('The Data would be stored in: ', store_pth)

    if save:
        with open(f'{store_pth}train_en.txt', 'w') as f:
            f.write(train_en)
        with open(f'{store_pth}train_vi.txt', 'w') as f:
            f.write(train_vi)
        with open(f'{store_pth}test_en.txt', 'w') as f:
            f.write(test_en)
        with open(f'{store_pth}test_vi.txt', 'w') as f:
            f.write(test_vi)
    
    print('Done!')
    


The Data would be stored in:  ./data/iwsltenvi/
Done!


# 1